In [9]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from matplotlib import pyplot  as plt

# Device configuration
device = 'cpu'#torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# 1 Этап - подготовка данных!

In [2]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


# 2 Этап - Нейронная сеть!

In [3]:
# Fully connected neural network with one hidden layer
class NeuralNet2020(nn.Module):
    def __init__(self, hidden_size1, hidden_size2, num_classes):
        super(NeuralNet2020, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2)
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(16 * 5 * 5, hidden_size1) # fully connected
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)  
        self.fc3 = nn.Linear(hidden_size2, num_classes)  
    
    def forward(self, x): # вход размера 1 x 28 x 28
        out = self.conv1(x) # на выходе размер 6 x 28 x 28
        out = self.maxpool(out) # на выходе размер 6 x 14 x 14
        out = self.relu(out)
        out = self.conv2(out)  # на выходе размер 16 x 10 x 10
        out = self.maxpool(out) # на выходе размер 16 x 5 x 5
        out = self.relu(out)
        out = out.reshape(-1, 16 * 5 * 5) # на выходе размер 400
        out = self.fc1(out) # на выходе размер 120
        out = self.relu(out)
        out = self.fc2(out) # на выходе размер 84
        out = self.relu(out)
        out = self.fc3(out) # на выходе размер 10
        
        return out

model = NeuralNet2020(hidden_size1=120, hidden_size2=84, num_classes=10).to(device)


In [7]:
f = nn.Sigmoid()

In [ ]:
plt.plot()

In [ ]:
nn.LeakyReLU
nn.Sigmoid
nn.Tanh
nn.ReLU6
nn.ReLU
nn

# 3 этап - Loss-функция и оптимизатор

In [5]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [1]:
torch.optim.

SyntaxError: invalid syntax (<ipython-input-1-44a3c561344c>, line 1)

In [5]:
for p in model.parameters():
    print(p)

Parameter containing:
tensor([[[[-0.0486, -0.1800, -0.0339,  0.1451, -0.1941],
          [ 0.0687, -0.0390,  0.1900, -0.0014,  0.1593],
          [ 0.1933, -0.0990, -0.1526, -0.1803,  0.0115],
          [-0.0654, -0.1232, -0.1043, -0.0863,  0.0032],
          [ 0.1002,  0.1634, -0.0918, -0.0895,  0.0322]]],


        [[[-0.0128,  0.1006,  0.0993,  0.1207,  0.1952],
          [ 0.0021,  0.1277,  0.1451,  0.0640,  0.1641],
          [-0.1097, -0.0304, -0.0489, -0.1725, -0.0127],
          [ 0.1010, -0.1251, -0.1002, -0.1340, -0.1207],
          [-0.1416, -0.0503,  0.1240, -0.0578, -0.0641]]],


        [[[ 0.1553,  0.0176, -0.0058,  0.0128, -0.1292],
          [ 0.1775,  0.0638, -0.1312, -0.1145, -0.0869],
          [-0.1531,  0.0495, -0.0912, -0.0144,  0.0954],
          [ 0.0828, -0.0469, -0.1722,  0.0610, -0.0800],
          [ 0.1036,  0.0977, -0.0879, -0.1154, -0.0385]]],


        [[[ 0.0452,  0.1234,  0.0238,  0.0653,  0.1563],
          [-0.0538,  0.0471, -0.1939,  0.0743, -0.0585

# 4 этап - обучение нейросети

In [6]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        #images = images.reshape(-1, 28*28).to(device)
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            #images = images.reshape(-1, 28*28).to(device)
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

    # Save the model checkpoint
    torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.5781
Epoch [1/5], Step [200/600], Loss: 0.1809
Epoch [1/5], Step [300/600], Loss: 0.1183
Epoch [1/5], Step [400/600], Loss: 0.0910
Epoch [1/5], Step [500/600], Loss: 0.1362
Epoch [1/5], Step [600/600], Loss: 0.0923
Accuracy of the network on the 10000 test images: 97.15 %
Epoch [2/5], Step [100/600], Loss: 0.1018
Epoch [2/5], Step [200/600], Loss: 0.1440
Epoch [2/5], Step [300/600], Loss: 0.2593
Epoch [2/5], Step [400/600], Loss: 0.0831
Epoch [2/5], Step [500/600], Loss: 0.0361
Epoch [2/5], Step [600/600], Loss: 0.0572
Accuracy of the network on the 10000 test images: 98.11 %
Epoch [3/5], Step [100/600], Loss: 0.1404
Epoch [3/5], Step [200/600], Loss: 0.0260
Epoch [3/5], Step [300/600], Loss: 0.0217
Epoch [3/5], Step [400/600], Loss: 0.0254
Epoch [3/5], Step [500/600], Loss: 0.0586
Epoch [3/5], Step [600/600], Loss: 0.0629
Accuracy of the network on the 10000 test images: 98.53 %
Epoch [4/5], Step [100/600], Loss: 0.0372
Epoch [4/5], Step [200/600],

In [14]:
ss = []
for p in model.parameters():
    ss.append(p.numel())
print(ss)
sum(ss)

[150, 6, 2400, 16, 48000, 120, 10080, 84, 840, 10]


61706